# 1. 동 및 업종별 기업 데이터 정리

In [3]:
import pandas as pd
import numpy as np

data = pd.read_csv('../김은선/서울_더존_빅벨류_업종별_동별_경위도.csv', encoding='utf-8-sig')
data.drop('Unnamed: 0', axis=1, inplace=True)
data.head()

,업종,ADDR,구,동,동도로명,서울시구,lon,lat
0,도매 및 소매업,퇴계로75길,중구,흥인동,흥인동퇴계로75길,서울특별시 중구,127.015582,37.566224
1,"전문, 과학 및 기술 서비스업",퇴계로75길,중구,흥인동,흥인동퇴계로75길,서울특별시 중구,127.015582,37.566224
2,부동산업,백제고분로48길,송파구,방이동,방이동백제고분로48길,서울특별시 송파구,127.115758,37.510815
3,"전문, 과학 및 기술 서비스업",백제고분로48길,송파구,방이동,방이동백제고분로48길,서울특별시 송파구,127.115758,37.510815
4,도매 및 소매업,백제고분로48길,송파구,방이동,방이동백제고분로48길,서울특별시 송파구,127.115758,37.510815


In [5]:
# 각 동에서의 업종별 기업 개수 정리
data_count = data.copy()
data_count['cnt'] = 1
data_count = pd.DataFrame(data_count.groupby(['동', '업종'])['cnt'].count())
data_count = data_count.add_suffix('_Count').reset_index()
data_count = data_count.sort_values('cnt_Count', ascending=False)
data_count.head()

# data_count.to_csv('../빅벨류_동별_업종 정리.csv', encoding='utf-8-sig')

,동,업종,cnt_Count
18,가산동,정보통신업,302
139,구로동,정보통신업,190
17,가산동,"전문, 과학 및 기술 서비스업",181
874,서초동,"전문, 과학 및 기술 서비스업",132
1167,역삼동,부동산업,115


In [7]:
# pivot 테이블식으로 동별 업종별 개수 확인
data_pivot = pd.pivot_table(data_count, index=['동'], values = ['cnt_Count'], columns = ['업종'], fill_value = 0)
data_pivot.columns = ['_'.join(col) for col in data_pivot.columns.values]
data_pivot.columns = ['교육 서비스업', '금융 및 보험업', '도매 및 소매업', '부동산업', '사업시설 관리, 사업지원 및 임대 서비스업', '전문, 과학 및 기술 서비스업', '정보통신업']
data_pivot.reset_index(inplace=True)
data_pivot.head()
# data_pivot.to_csv('../동별_업종개수정리.csv', encoding='utf-8-sig')

,동,교육 서비스업,금융 및 보험업,도매 및 소매업,부동산업,"사업시설 관리, 사업지원 및 임대 서비스업","전문, 과학 및 기술 서비스업",정보통신업
0,가락동,5,1,46,21,8,7,5
1,가리봉동,0,0,4,5,3,1,6
2,가산동,4,4,35,40,13,181,302
3,가양동,0,1,13,10,1,2,2
4,가회동,0,0,0,1,0,4,0


# 2. 동별 입지추천용 데이터 준비하기

In [8]:
fin = data_pivot.copy()

In [33]:
features=pd.read_csv('/home/jovyan/WIDE_HOME/Null제거_dataset2.csv')
features.drop('Unnamed: 0', axis=1, inplace=True)
features.drop('Unnamed: 0.1.1', axis=1, inplace=True)
features.sample(3)

,년월,지번 주소 코드,지번주소+동 코드,대장구분 코드,주소,법정동 구 코드,법정동 동 코드,대지구분,번지1,번지2,...,동,사업체 창업률,교통매력도,주택매력도,직업매력도,총생활인구,청소년생활인구,청년생활인구,장년생활인구,사업체밀도
1877,202105,1165010100108110006,BV1165010100108110006000,1,서초구 방배동,11650,10100,1.0,811,6,...,방배동,12.69,0.140904,0.105377,0.071508,4.085443e+06,714904.5,536236.687500,643660.479167,1592.750000
90,202105,1123010500105300016,BV1123010500105300016000,1,동대문구 답십리동,11230,10500,1.0,530,16,...,답십리동,13.07,0.258026,0.106312,0.115119,4.667641e+06,684244.5,581257.500000,734306.500000,2503.000000
1454,202105,1168010600110090000,BV1168010600110090000000,1,강남구 대치동,11680,10600,1.0,1009,0,...,대치동,12.76,0.258026,0.106312,0.115119,5.492734e+06,1639957.5,655365.333333,846067.111111,2779.333333


In [28]:
# 필요한 칼럼 확인
features.columns[43:55]

Index(['법정동명', '평당금액2', '동', '사업체 창업률', '교통매력도', '주택매력도', '직업매력도', '총생활인구',
       '청소년생활인구', '청년생활인구', '장년생활인구', '사업체밀도'],
      dtype='object')

In [29]:
# 필요 칼럼만 가져오기
features = features[features.columns[43:55]]

# 중복 데이터 제거
features = features.drop_duplicates()
features.reset_index(inplace=True, drop=True)

# 동 칼럼 제거
features.drop('동', axis=1, inplace=True)

# features데이터는 법정동명 기준으로 합치기.
fin_data = pd.merge(fin, features, how='right', left_on='동', right_on = '법정동명')
fin_data = fin_data.dropna()

In [31]:
fin_data.drop_duplicates('동', keep='first', inplace=True)
fin_data
# fin_data.to_csv('동별_입지추천_1번.csv', encoding='utf-8-sig')

,동,교육 서비스업,금융 및 보험업,도매 및 소매업,부동산업,"사업시설 관리, 사업지원 및 임대 서비스업","전문, 과학 및 기술 서비스업",정보통신업,법정동명,평당금액2,사업체 창업률,교통매력도,주택매력도,직업매력도,총생활인구,청소년생활인구,청년생활인구,장년생활인구,사업체밀도
0,가락동,5.0,1.0,46.0,21.0,8.0,7.0,5.0,가락동,11385493,12.740000,0.116528,0.101555,0.062677,4.446749e+06,765395.571429,5.605293e+05,7.307886e+05,2105.000000
1,가산동,4.0,4.0,35.0,40.0,13.0,181.0,302.0,가산동,7316824,16.210000,0.532699,0.219801,0.625279,3.261598e+06,170662.250000,7.480468e+05,5.980416e+05,6612.000000
2,가양동,0.0,1.0,13.0,10.0,1.0,2.0,2.0,가양동,10897041,12.850000,0.189061,0.114424,0.165897,1.042913e+06,125929.000000,1.606970e+05,1.835663e+05,926.000000
3,갈월동,0.0,0.0,0.0,1.0,0.0,0.0,1.0,갈월동,12545719,15.674717,0.255648,0.134810,0.143906,2.763232e+06,272339.500000,4.802420e+05,4.870367e+05,2023.306604
8,갈현동,0.0,0.0,15.0,8.0,3.0,4.0,1.0,갈현동,6066586,12.890000,0.312723,0.161060,0.211805,6.673195e+06,974200.428571,1.049023e+06,1.001296e+06,1594.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,효창동,1.0,0.0,8.0,3.0,0.0,1.0,2.0,효창동,10851096,11.480000,0.090369,0.040902,0.038456,6.057872e+06,901723.000000,9.770270e+05,9.867423e+05,950.000000
416,후암동,0.0,1.0,8.0,6.0,1.0,3.0,1.0,후암동,9164659,15.730000,0.101070,0.137615,0.070651,3.362217e+06,423956.000000,4.840460e+05,5.634857e+05,1079.000000
417,휘경동,2.0,0.0,8.0,6.0,0.0,0.0,0.0,휘경동,8351569,13.260000,0.227111,0.069827,0.108809,4.297992e+06,530672.200000,1.056058e+06,5.821961e+05,1319.000000
419,흑석동,1.0,0.0,3.0,2.0,1.0,0.0,0.0,흑석동,15165395,13.870000,0.211653,0.133792,0.093904,2.693486e+06,340875.833333,5.578828e+05,3.858713e+05,927.000000


In [32]:
# 확인
fin_data['동'].value_counts()

교남동      1
순화동      1
원남동      1
반포동      1
세곡동      1
        ..
쌍문동      1
의주로1가    1
경운동      1
명륜3가     1
연희동      1
Name: 동, Length: 238, dtype: int64